In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 2
import sys
import traceback # for stack trace formatting
import importlib
from pathlib import Path
import os

In [5]:
import re
from enum import Enum

script_dir = Path('scripts') # Path(os.path.dirname(os.path.abspath(__file__))) # /home/halechr/repos/Spike3D/scripts
print(f'script_dir: {script_dir}')
root_dir = script_dir.parent # Spike3D root repo dir
os.chdir(root_dir)

In [6]:
def insert_text(source_file, insert_text_str:str, output_file, insertion_string:str='<INSERT_HERE>'):
    """Inserts the text from insert_text_str into the source_file at the insertion_string, and saves the result to output_file.

    Args:
        source_file (_type_): _description_
        insert_text_str (str): _description_
        output_file (_type_): _description_
        insertion_string (str, optional): _description_. Defaults to '<INSERT_HERE>'.
    """
    # Load the source text
    with open(source_file, 'r') as f:
        source_text = f.read()

    # Find the insertion point in the source text
    insert_index = source_text.find(insertion_string)

    # Insert the text
    updated_text = source_text[:insert_index] + insert_text_str + source_text[insert_index:]

    # Save the updated text to the output file
    with open(output_file, 'w') as f:
        f.write(updated_text)

def insert_text_from_file(source_file, insert_file, output_file, insertion_string:str='<INSERT_HERE>'):
    """ Wraps insert_text, but loads the insert_text from a file instead of a string. """
    # Load the insert text
    with open(insert_file, 'r') as f:
        insert_text_str = f.read()
    insert_text(source_file, insert_text_str, output_file, insertion_string)


# ==================================================================================================================== #
# Project versioning:                                                                                                  #
# ==================================================================================================================== #

# pyproject_files = {'release':'pyproject_release.toml', 'dev':'pyproject_dev.toml'}

class VersionType(Enum):
    """Docstring for VersionType."""
    RELEASE = "release"
    dev = "dev"
    
    @property
    def pyproject_template_file(self):
        """The pyproject_exclusive_text property."""
        filename = {'release': "templating/pyproject_template_release.toml_fragment",
        'dev': "templating/pyproject_template_dev.toml_fragment"}
        if self.name == VersionType.RELEASE.name:
            return filename['release']
        elif self.name == VersionType.dev.name:
            return filename['dev']
        else:
            raise ValueError(f"VersionType {self.name} not recognized.")

    @property
    def pyproject_exclusive_text(self):
        """The pyproject_exclusive_text property."""
        _pyproject_exclusive_text_dict = {'release': """[tool.poetry.group.local.dependencies]
        neuropy = {path = "../NeuroPy", develop=true} # , extras = ["acceleration"]
        pyphocorehelpers = {path = "../pyPhoCoreHelpers", develop=true}
        pyphoplacecellanalysis = {path = "../pyPhoPlaceCellAnalysis", develop=true}""",
        'dev': """[tool.poetry.group.local.dependencies]
        neuropy = {path = "../NeuroPy", develop=true} # , extras = ["acceleration"]
        pyphocorehelpers = {path = "../pyPhoCoreHelpers", develop=true}
        pyphoplacecellanalysis = {path = "../pyPhoPlaceCellAnalysis", develop=true}"""}
        if self.name == VersionType.RELEASE.name:
            return _pyproject_exclusive_text_dict['release']
        elif self.name == VersionType.dev.name:
            return _pyproject_exclusive_text_dict['dev']
        else:
            raise ValueError(f"VersionType {self.name} not recognized.")
    
    @classmethod
    def init_from_is_release(cls, is_release: bool):
        if is_release:
            return cls.RELEASE
        else:
            return cls.dev

In [191]:
def replace_text_in_file(file_path, regex_pattern, replacement_string, debug_print=False):
    with open(file_path, 'r') as file:
        file_content = file.read()

    if debug_print:
        print(f"====================== Read from file ({file_path}) ======================:\n{file_content}")
    
    # updated_content = re.sub(regex_pattern, replacement_string, file_content, flags=re.MULTILINE)
    target_replace_strings = re.findall(regex_pattern, file_content, re.MULTILINE)
    assert len(target_replace_strings) == 1
    target_replace_string = target_replace_strings[0]
    if debug_print:
        print(f'Replacing:\n{target_replace_string}')
        print(f"====================== replacing ======================:\n{target_replace_string}\n\n====================== with replacement string ====================== :\n{replacement_string}\n\n")
    updated_content = file_content.replace(target_replace_string, replacement_string, 1)
    if debug_print:
        print(updated_content)

    if debug_print:
        print(f"======================  updated_content ====================== :\n{updated_content}\n\n")
        print(f"====================== saving to {file_path}...")
    with open(file_path, 'w') as file:
        file.write(updated_content)

In [192]:
def build_pyproject_toml_file(repo_path, is_release=False, pyproject_template_file_name = 'templating/pyproject_template.toml_template', pyproject_final_file_name = 'pyproject.toml', debug_print=True):
    """ Builds the complete final pyproject.toml file from the pyproject_template.toml_template for the current version (release or dev)

    from Spike3D.scripts.setup_dependent_repos import build_pyproject_toml_file
    build_pyproject_toml_file("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis")

    """
    os.chdir(repo_path)
    curr_version = VersionType.init_from_is_release(is_release)
    if debug_print:
        print(f'Templating: Building pyproject.toml for {curr_version.name} version in {repo_path}...')
        # insert_text(pyproject_template_file_name, curr_version.pyproject_exclusive_text, pyproject_final_file_name, insertion_string='<INSERT_HERE>')
        print(f"\tpyproject_template_file_name: {pyproject_template_file_name},\n\tcurr_version.pyproject_template_file: {curr_version.pyproject_template_file},\n\tpyproject_final_file_name: {pyproject_final_file_name},\n\tinsertion_string='<INSERT_HERE>'")
        # insert_text_from_file(pyproject_template_file_name, curr_version.pyproject_template_file, pyproject_final_file_name, insertion_string='<INSERT_HERE>')

    # remote_dependencies_regex = r"^\[tool\.poetry\.group\.remote\.dependencies\]\n((?:.+\n)+?)\n"
    # remote_dependencies_regex = r"^(\s*\[tool\.poetry\.group\.remote\.dependencies\]\n(?:.+\n)*\n)"
    remote_dependencies_regex = r"^(\s*\[tool\.poetry\.group\.remote\.dependencies\]\n(?:.+\n)*)\n\[?"
    # Load the insert text
    with open(curr_version.pyproject_template_file, 'r') as f:
        insert_text_str = f.read()
        
    if not insert_text_str.startswith('\n'):
        # Add a leading newline if the loaded text doesn't already have one
        insert_text_str = '\n' + insert_text_str
    if not insert_text_str.endswith('\n\n'):
        # Add a trailing newline if the loaded text doesn't already have one
        insert_text_str = insert_text_str + '\n'
    
    if debug_print:
        print(insert_text_str)
    
    replace_text_in_file(pyproject_final_file_name, remote_dependencies_regex, insert_text_str, debug_print=debug_print)

In [138]:
remote_dependencies_regex = r"^\[tool\.poetry\.group\.remote\.dependencies\]\n((?:.+\n)+?)\n"
pyproject_template_file_name = 'templating/pyproject_template.toml_template'
pyproject_final_file_name = 'pyproject.toml'


In [193]:
test_repo_path = Path("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis")
# test_repo_path = Path("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers")

In [194]:
build_pyproject_toml_file(test_repo_path, is_release=False, debug_print=False)

In [164]:
file_path = str(Path(test_repo_path).joinpath(pyproject_final_file_name).resolve())
file_path

'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoCoreHelpers\\pyproject.toml'

In [165]:
with open(file_path, 'r') as file:
    file_content = file.read()
print(file_content)

[tool.poetry]
name = "pyphocorehelpers"
version = "0.0.2"
description = "A collection of generally useful python code by Pho Hale"
authors = ["Pho Hale <CommanderPho@users.noreply.github.com>"]
readme = "README.md"

[tool.poetry.dependencies]
python = ">=3.9,<3.11"
objsize = "^0.6.1"
indexed = "^1.3.0"
PyQt5Singleton = "^0.1"
h5py = "^3.8.0"
QtPy = "^2.1.0"
PyQt5-sip = "^12.11.1"
numpy = "^1.23.2"
pandas = "^1.5.1"

ansi2html = "^1.8.0"
numba = "^0.56.4"

[tool.poetry.group.dev.dependencies]
importlib-metadata = "^6.0.0"


[tool.poetry.group.viz.dependencies]
pyvista = "^0.38.0"
matplotlib = "^3.6.3"
vedo = "^2023.4.3"
hdf5plugin = "^4.1.1"
silx = "^1.1.2"
vtk = "^9.2.5"

[tool.poetry.group.remote.dependencies]
neuropy = {git = "https://github.com/CommanderPho/NeuroPy.git"} # , rev = "main", extras = ["acceleration"]

[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"



In [166]:
is_release = True
curr_version = VersionType.init_from_is_release(is_release)
# remote_dependencies_regex = r"^\[tool\.poetry\.group\.remote\.dependencies\]\n((?:.+\n)+?)\n"
# remote_dependencies_regex = r"^(\s*\[tool\.poetry\.group\.remote\.dependencies\]\n(?:.+\n)*\n)"
remote_dependencies_regex = r"^(\s*\[tool\.poetry\.group\.remote\.dependencies\]\n(?:.+\n)*)\n\[?"
# Load the insert text
with open(curr_version.pyproject_template_file, 'r') as f:
    insert_text_str = f.read()
if not insert_text_str.startswith('\n'):
    # Add a leading newline if the loaded text doesn't already have one
    insert_text_str = '\n' + insert_text_str
if not insert_text_str.endswith('\n\n'):
    # Add a trailing newline if the loaded text doesn't already have one
    insert_text_str = insert_text_str + '\n'
print(insert_text_str)


[tool.poetry.group.remote.dependencies]
neuropy = {git = "https://github.com/CommanderPho/NeuroPy.git"} # , rev = "main", extras = ["acceleration"]



In [167]:
test_remote_dependencies_regex = remote_dependencies_regex
# test_remote_dependencies_regex = r'^\\[tool\\.poetry\\.group\\.remote\\.dependencies\\]\\n((?:.+\n)+?)\\n'
# test_remote_dependencies_regex = r"\n^\s*\[tool\.poetry\.group\.remote\.dependencies\]\n((?:\s+.+\n)+)"
# test_remote_dependencies_regex = r"^(\s*\[tool\.poetry\.group\.remote\.dependencies\]\n(?:.+\n)*\n)"

re.findall(pattern=test_remote_dependencies_regex, string=file_content, flags=re.MULTILINE)

['\n[tool.poetry.group.remote.dependencies]\nneuropy = {git = "https://github.com/CommanderPho/NeuroPy.git"} # , rev = "main", extras = ["acceleration"]\n']

In [176]:
target_replace_strings = re.findall(test_remote_dependencies_regex, file_content, re.MULTILINE)
assert len(target_replace_strings) == 1
target_replace_string = target_replace_strings[0]
target_replace_string

updated_content = file_content.replace(target_replace_string, insert_text_str, 1)
print(updated_content)

In [173]:
re.match(test_remote_dependencies_regex, file_content, re.MULTILINE)

In [169]:
updated_content = re.sub(test_remote_dependencies_regex, insert_text_str, file_content, flags=re.MULTILINE)
print(updated_content)

[tool.poetry]
name = "pyphocorehelpers"
version = "0.0.2"
description = "A collection of generally useful python code by Pho Hale"
authors = ["Pho Hale <CommanderPho@users.noreply.github.com>"]
readme = "README.md"

[tool.poetry.dependencies]
python = ">=3.9,<3.11"
objsize = "^0.6.1"
indexed = "^1.3.0"
PyQt5Singleton = "^0.1"
h5py = "^3.8.0"
QtPy = "^2.1.0"
PyQt5-sip = "^12.11.1"
numpy = "^1.23.2"
pandas = "^1.5.1"

ansi2html = "^1.8.0"
numba = "^0.56.4"

[tool.poetry.group.dev.dependencies]
importlib-metadata = "^6.0.0"


[tool.poetry.group.viz.dependencies]
pyvista = "^0.38.0"
matplotlib = "^3.6.3"
vedo = "^2023.4.3"
hdf5plugin = "^4.1.1"
silx = "^1.1.2"
vtk = "^9.2.5"

[tool.poetry.group.remote.dependencies]
neuropy = {git = "https://github.com/CommanderPho/NeuroPy.git"} # , rev = "main", extras = ["acceleration"]
build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"



In [48]:
import re

text = """
[tool.poetry.group.remote.dependencies]
requests = "^2.25.1"
numpy = "^1.20.1"
"""

regex = r"^\[tool\.poetry\.group\.remote\.dependencies\]\n((?:.+\n)+?)\n"
matches = re.findall(regex, text, re.MULTILINE)

print(matches)


[]


In [51]:
import re

text = """
  [tool.poetry.group.remote.dependencies]
  requests = "^2.25.1"
  numpy = "^1.20.1"
"""

regex = r"\n^(\s*\[tool\.poetry\.group\.remote\.dependencies\]\n(?:\s*.+\s*\n)+)"
matches = re.findall(regex, text, re.MULTILINE)

print(matches)


['  [tool.poetry.group.remote.dependencies]\n  requests = "^2.25.1"\n  numpy = "^1.20.1"\n']
